Set Up the Selenium server in order to access ChromeDriver

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pyarrow.parquet as pq


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

df_links = pq.read_table("list of company websites.snappy.parquet").to_pandas()

# df_links.rename(columns={"max(page)": "initial_link"}, inplace=True)
df_links


,domain
0,umbrawindowtinting.com
1,embcmonroe.org
2,caffeygroup.com
3,sk4designs.com
4,draftingdesign.com
...,...
2474,northernirelandscreen.co.uk
2475,birdbrand.co.uk
2476,thorntonlodgenorthyorkshire.co.uk
2477,peasholmecharity.org.uk


In [11]:
# add https protocol to the links
def add_protocol(link):
    if link.startswith("http"):
        return link
    else:
        return "https://" + link
for i in range(len(df_links)):
    df_links.loc[i, "domain"] = add_protocol(df_links.loc[i, "domain"])

In [22]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def scraping_text(url):
    driver.get(url)
    text = driver.find_element(By.XPATH, "/html/body").text
    return text

def scraping_links_of_interest(url):
    driver.get(url)
    elements = driver.find_elements(By.XPATH, "//a[@href]")

    links = [element.get_attribute("href") for element in elements]
    links_of_interest = list(filter(lambda x: "product" in x, links))
    print(links_of_interest)
    return links_of_interest

def append_link_of_interest(index,url):
    links = scraping_links_of_interest(url)
    df_links.at[index,'links_of_interest'] = links


for index,row in df_links.iterrows(): 
    print(row['domain'])   
    # append_link_of_interest(index,row['domain'])

df_links.head()
df_links.to_csv("agregate_pages.csv", index=False)
driver.quit()

https://umbrawindowtinting.com
https://embcmonroe.org
https://caffeygroup.com
https://sk4designs.com
https://draftingdesign.com
https://truesdail.com
https://seedsourceag.com
https://romebeerfest.com
https://beerock.com
https://cabwhp.org
https://saintmlc.com
https://dillonmusic.com
https://societyfortheblind.org
https://perfectsearchinc.com
https://wyandottewinery.com
https://thespiritofblackjackmountain.com
https://kingdomtel.com
https://triadstage.org
https://savagecbd.com
https://clubk-9.com
https://katerisyracuse.com
https://triplecrownconstruction.com
https://greyhackle.com
https://wildandwanderful.com
https://coastalpaddlecompany.com
https://casepaper.com
https://searchouthomes.com
https://richardbrothersfinancial.com
https://paramountlandtitle.com
https://villageatfourhills.com
https://aiwoodwork.com
https://infometrica.com
https://twinpondsnashua.com
https://oauconsulting.com
https://neutresalon.com
https://glacier.chat
https://systemadix.com
https://your-doctor.org
https://se

In [23]:
df_links = pd.read_csv("agregate_pages.csv")
for index,row in df_links.head(1).iterrows():
        text = scraping_text(row['domain'])
        df_links.at[index,"text"] = text

df_links.head()

MaxRetryError: HTTPConnectionPool(host='localhost', port=45383): Max retries exceeded with url: /session/792eff18ba36118e3059dc4b65d9fbd6/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d30329d1b10>: Failed to establish a new connection: [Errno 111] Connection refused'))

def data_for_training():
    directory = "./text_from_links"
    product_labels = product_label()
    count = 0
    for file in os.listdir(directory):
        if file.endswith(".txt"):
            texts = []
            print(f"Reading {file}")
            with open(f"{directory}/{file}", "r") as f:
                text = f.read()
            text = text.lower()
            for line in text.split("\n"):
                if line:
                    for label in product_labels:
                        if label in line:
                            texts.append(line)
                            break
            with open(f"training_data.txt", "a") as f:
                f.write('\n'.join(texts))
            count += 1
            if count == 100:
                break